In [2]:
import numpy as np
import pandas as pd
import tweepy # pip install teepy should do (or conda mayhaps)

In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Part one -----------------------------------------------------------------------
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Making the twitter profile list

In [3]:
#Get the csv from the Sunlight Foundation
legs = pd.read_csv("/home/polichinel/Documents/KU/7.semester/vkm/legislators.csv") # my old folder
print(len(legs))

953


In [4]:
legs.columns

Index(['title', 'firstname', 'middlename', 'lastname', 'name_suffix',
       'nickname', 'party', 'state', 'district', 'in_office', 'gender',
       'phone', 'fax', 'website', 'webform', 'congress_office', 'bioguide_id',
       'votesmart_id', 'fec_id', 'govtrack_id', 'crp_id', 'twitter_id',
       'congresspedia_url', 'youtube_url', 'facebook_id', 'official_rss',
       'senate_class', 'birthdate', 'oc_email'],
      dtype='object')

In [5]:
# getting the twitter culunm - still including NaNs
twitter_prof_w_NaN = legs.twitter_id 
print(twitter_prof_w_NaN.isna().any())

True


In [6]:
# Creating list with twitter profiles - no NaNs
twitter_prof=[]

for i in twitter_prof_w_NaN:
    if str(i) != "nan":
        twitter_prof.append(i)

print("number of twitter profiles: " + str(len(twitter_prof))) #Number of profiles
print("first five: " + str(twitter_prof[0:5])) #Head; first five

number of twitter profiles: 535
first five: ['Robert_Aderholt', 'SenAlexander', 'MarkAmodeiNV2', 'RepAdams', 'reppeteaguilar']


In [7]:
# Adding the @ in front of the names - not sure it is needed
att_twitter_prof = []

for i in twitter_prof:
    k = "@" + i
    att_twitter_prof.append(k)
    
print("first five: " + str(att_twitter_prof[0:5])) 

first five: ['@Robert_Aderholt', '@SenAlexander', '@MarkAmodeiNV2', '@RepAdams', '@reppeteaguilar']


In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Part two -----------------------------------------------------------------------
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Scraping the tweets

In [8]:
consumer_key =  # insert your own from the twitter app site
consumer_secret =  # insert your own  from the twitter app site

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify = True)

In [13]:
# the scraping -----------------------------------------------------------------------

one_user_data = []
one_user_name = []

x=0

for legislator in att_twitter_prof[0:1]: # here, now, I just take the first one on the list. Change to [0:] to take all.
    try:
        for tweet in tweepy.Cursor(api.friends, id=legislator).items(10): # 10 = the number of freinds to get per legislature
            one_user_data.append(tweet)
            one_user_name.append(legislator) # for groupvariable - could be made a tuple wit the above, but this is easy...
            x=x+1 #just a counter to asses process
            print(x, end=' ') # printing horizontal loadingline (the counter)
    except(RuntimeError, TypeError, NameError, ZeroDivisionError, IndexError, tweepy.TweepError) as e:
        print(e, end=' ') # printing the name of the error
        continue

# Note that if you hit the rate limit (you properly will) the scarping will just wait (15 minunte ish) and then start agian.
# Change api.friends to api.user_timeline to get timeline. For more API's see tweepy's documentation.
# the scraping -----------------------------------------------------------------------

1 2 3 4 5 6 7 8 9 10 

In [16]:
# Just checking the mess
one_user_data

[User(_api=<tweepy.api.API object at 0x7f3abb796ac8>, _json={'id': 36771809, 'id_str': '36771809', 'name': 'NationalParkService', 'screen_name': 'NatlParkService', 'location': '417 locations', 'description': "America's Best Idea. #FindYourPark RT/follow/likes≠endorsement", 'url': 'https://t.co/KHyfn2YXhY', 'entities': {'url': {'urls': [{'url': 'https://t.co/KHyfn2YXhY', 'expanded_url': 'http://www.nps.gov', 'display_url': 'nps.gov', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 475733, 'friends_count': 2724, 'listed_count': 5902, 'created_at': 'Thu Apr 30 21:03:01 +0000 2009', 'favourites_count': 4675, 'utc_offset': -14400, 'time_zone': 'Eastern Time (US & Canada)', 'geo_enabled': True, 'verified': True, 'statuses_count': 9234, 'lang': 'en', 'status': {'created_at': 'Fri Apr 20 16:25:39 +0000 2018', 'id': 987366695332401154, 'id_str': '987366695332401154', 'text': 'RT @Interior: What did one marine mammal say to the other? \n\n"That\'s the 

In [18]:
# A quick dataset constructed on the basis of some of the mess above

def process_one_user(one_user_data):
    id_list = [tweet.id for tweet in one_user_data]
    data_set = pd.DataFrame(id_list, columns=["id"])
    
    data_set["Name_of_legislator"] = one_user_name
    data_set["Name_of_friend"] = [tweet.name for tweet in one_user_data]
    data_set["Screen_name_of_friend"] = [tweet.screen_name for tweet in one_user_data]
    data_set["location_of_friend"] = [tweet.location for tweet in one_user_data]
    data_set["description_of_friend"] = [tweet.description for tweet in one_user_data]

    return data_set
data_set = process_one_user(one_user_data)
# there is probaly loads more data to get from the string above

In [19]:
# Just a check
data_set

,id,Name_of_legislator,Name_of_friend,Screen_name_of_friend,location_of_friend,description_of_friend
0,36771809,@Robert_Aderholt,NationalParkService,NatlParkService,417 locations,America's Best Idea. #FindYourPark RT/follow/l...
1,2825569022,@Robert_Aderholt,Marshall Republican,marshallcoalgop,"Marshall County, Alabama",Marshall County Alabama Republican Club - Meet...
2,159467804,@Robert_Aderholt,Alabama YRs,YRFAL,Alabama,Young Republican Federation of Alabama!
3,1202616420,@Robert_Aderholt,Alabama Archives,AlabamaArchives,"Montgomery, Alabama",Find your story at the home of Alabama history!
4,88831888,@Robert_Aderholt,Madison CoAL GOP🇺🇸,4gop,"Huntsville, Alabama","The Madison County, Alabama Republican Party #..."
5,123577910,@Robert_Aderholt,Speaker McCutcheon,MacDistrict25,"Huntsville, AL",Alabama's Speaker of the House. Proudly servin...
6,818927131883356161,@Robert_Aderholt,Sarah Sanders,PressSec,"Washington, D.C.",@WhiteHouse Press Secretary. Proudly represent...
7,22772396,@Robert_Aderholt,AL200,al200,"Montgomery, AL","Between 2017 and 2019, the Alabama Bicentennia..."
8,55562722,@Robert_Aderholt,Redstone Arsenal,TeamRedstone,"Huntsville, AL","Official Twitter for Redstone Arsenal, AL. (Fo..."
9,52770481,@Robert_Aderholt,Sweet Home Alabama,TweetHomeAla,Alabama,The official feed of the Alabama Tourism Depar...


In [53]:
data_set.to_csv("friends.csv")